In [1]:
import sys
sys.path.append("..")
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
from scipy import optimize
from writeit import rewritePipes

Populating the interactive namespace from numpy and matplotlib


In [2]:
case = 1

D = .1            #pipe diameter (m)
Mr = 0.007        #manning roughness coeffs
T = 24           #simulation time (s)
a = 100            #pressure wavespeed
Ltot = 48

if case ==0:
    Np = 4
    Nn = Np+1
    oldinp = "../indata/DFDpleasework5.inp"

if case ==1:
    Np =6
    Nn = Np+1
    oldinp = "../indata/DFDpleasework7.inp"

if case ==2:
    Np = 8
    Nn = Np+1
    oldinp = "../indata/DFDpleasework4.inp"

Ndof = Nn-3
    
L = Ltot/float(Np)
N = int(L)
dx = L/float(N)
M = max(int(T*a/(dx*.8))*5,1)
Qin =0.0087
Ain =  0.00793018096379

fn = "../indata/optim_elev_test"
params= (D,N,L,Mr,T,M,a,Qin,Ain,Np,fn,oldinp)
print M

15000


In [3]:
def f(z,*params):
    #e0,e1,e2,e3,e4,e5 = z
    #elevs = [10,e0,e1,e2,6,e3,e4,e5,10]#junction elevations
    e0,e1,e2,e3 = z
    elevs = [10,e0,e1,6,e2,e3,10]
    #e0,e1 = z
    #elevs = [10,e0,6,e1,10]#
    h0 =0.00328335033016
    Nn = len(elevs)
    D,N,L,Mr,T,M,a,Qin,Ain,Np,fn,oldinp = params
    dt = T/float(M)
    Ns = [N]*Np
    Ls = [L]*Np
    Mrs = [Mr]*Np  #manning roughness coeffs
    Ds = [D]*Np      #pipe diameter (m) 
    h0s = [h0]*Np    #IC for h
    q0s = [0]*Np    #IC for Q
    jt = [1]+[2]*(Nn-2)+[1]  #for DFDpleasework2.inp
    bt = [1]*Nn
    bv = [0]+[1]*(Nn-1)
    r =  [0]+[1]*(Nn-2)+[-1]
    
    (fi, fc) = rewritePipes(fn,oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, a, elevs)
    n1 = PyNetwork(fi,fc,1)
    Q00 = Qin*np.ones(M+1)
    #A00 =Ain*np.ones(M+1)
    n1.setbVal(0,Q00)
    #A0 = Ain*np.ones(Ns[0])
    #Q0 = np.zeros(Ns[0])
    #n1.setIC(0,A0,Q0)
    #Ain = 0.0087*ones(M+1)
    #A00 = 0.01*D*D*pi/4.*np.ones(Ns[0])
    #Q00 = np.zeros(Ns[0])
    #for k in range(0,Np):
    #    n1.setIC(k,A00,Q00)
    n1.runForwardProblem(dt)
    H = sum([n1.getAveGradH(i) for i in range(M+1)])
    #print "elevs = [10,%f, %f, %f, 6, %f, %f, 10] and H = %f"%(e0,e1,e2,e3,e4,H)
    return H

In [5]:
z0 = (7,8)
z0 = (7.5,6.5,7.5,8.5)
z0 = (8.5,7.5,8.5,9.5)

#z0 = (9,10,8,7,8,10)
#z0 = (7,7,7,8,8,8)
#z0  = (10,7,8,11,8.5,8,7.5)
#z0 = (9,5,7,11,10,9)
#z0 =(8,9,6,5,7,9)
#z0 = (9,8,7,7,8, 9)
#z0 = (8.5,9,7.5,8,9,11)
#z0 = 10

f(z0,*params)

620155.43193668046

In [6]:
if case==0:
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] -4},
        {'type': 'ineq', 'fun': lambda x: -x[0] +12},
        {'type': 'ineq', 'fun': lambda x:  x[1] -4},
        {'type': 'ineq', 'fun': lambda x: -x[1] +12},
       )
elif case==1:
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] -4},
        {'type': 'ineq', 'fun': lambda x: -x[0] +12},
        {'type': 'ineq', 'fun': lambda x:  x[1] -4},
        {'type': 'ineq', 'fun': lambda x: -x[1] +12},
        {'type': 'ineq', 'fun': lambda x:  x[2] -4},
        {'type': 'ineq', 'fun': lambda x: -x[2] +12},
        {'type': 'ineq', 'fun': lambda x:  x[3] -4},
        {'type': 'ineq', 'fun': lambda x: -x[3] +12},
       )
elif case==2:
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] -4},
        {'type': 'ineq', 'fun': lambda x: -x[0] +12},
        {'type': 'ineq', 'fun': lambda x:  x[1] -4},
        {'type': 'ineq', 'fun': lambda x: -x[1] +12},
        {'type': 'ineq', 'fun': lambda x:  x[2] -4},
        {'type': 'ineq', 'fun': lambda x: -x[2] +12},
        {'type': 'ineq', 'fun': lambda x:  x[3] -4},
        {'type': 'ineq', 'fun': lambda x: -x[3] +12},
        {'type': 'ineq', 'fun': lambda x:  x[4] -4},
        {'type': 'ineq', 'fun': lambda x: -x[4] +12},
        {'type': 'ineq', 'fun': lambda x:  x[5] -4},
        {'type': 'ineq', 'fun': lambda x: -x[5] +12},
       )

In [7]:
class MyBounds(object):
    def __init__(self, xmax=[12]*Ndof, xmin=[4]*Ndof):
        self.xmax = np.array(xmax) 
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [8]:
def print_f(x,f,accepted):
    print "f = %f x = %s, accepted = %d"%(f,str(x), int(accepted))

In [ ]:
minimizer_kwargs = {"method": "COBYLA","constraints":cons,"args":params}
np.random.seed(555)
mybounds = MyBounds()
ret = optimize.basinhopping(f, z0,  minimizer_kwargs=minimizer_kwargs,
                   niter=5, accept_test = mybounds, callback=print_f)

#res = optimize.anneal(f, z, args=params, schedule='boltzmann',
#                          full_output=True, maxiter=50, lower=20,
#                          upper=250, dwell=50, disp=True, feps = 1e-3)

In [ ]:
ret


In [ ]:
f0 = f(z0,*params)
ff = ret.fun
print T

In [ ]:
print "Np = &%d"%Np
print "Nfev = &%d"%ret.nfev
print "z_0= &%s"%(str(list(z0)))
print "z_f =& %s"%(str(list(ret.x)))
print "f_0 =& %f, f_f = %f, f_f/f_0=%f"%(f0,ff,ff/f0)

In [ ]:
print "z_0&= %s"%(str(z0))
print "z_f& = %s"%(str(ret.x))
print "f_0& = %f, f_f = %f, f_f/f_0=%f"%(f0,ff,ff/f0)

In [ ]:

Np = &6
Nfev = &340
z_0= &[7.5, 6.5, 7.5, 8.5]
z_f =& [8.7409181741445288, 6.4482813556630605, 7.8603789169278508, 7.8071324922307346]
f_0 =& 625689.371403, f_f = 110665.153922, f_f/f_0=0.176869

In [14]:
ret.x

array([ 8.93812713,  7.63065119,  6.71595473,  6.9738486 ,  7.95737693,
        9.0219969 ])